In [21]:
import pandas as pd
import sqlite3
import numpy as np
# Importing necessary libraries from sklearn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

In [29]:
# importing soccer data
conn = sqlite3.connect("database.sqlite")

# Reading Player_Attributes table to dataframe

player_attr_df = pd.read_sql("SELECT strength, stamina, jumping FROM Player_Attributes", conn)

# Filling with 11 for all null values
player_attr_df.fillna(11, inplace=True)

x = player_attr_df[['strength', 'stamina']].values
y = player_attr_df[['jumping']].values
X_train, X_test, y_train, y_test= train_test_split(x, y, test_size=0.3, random_state=0)

In [30]:
desicion_tree_params_grid = {'criterion':['gini','entropy'], 'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50], 'splitter':["best", "random"], 'random_state':[0,1,2,4,6,8,10,12,14,16,20,40,42]}
grid_search_decision_tree_classifier = GridSearchCV(DecisionTreeClassifier(), desicion_tree_params_grid, cv=10)
grid_search_decision_tree_classifier.fit(X_train, y_train)

print("Decision Tree best grid score: " + str(grid_search_decision_tree_classifier.best_score_))
print("Decision Tree grid test score: " + str(grid_search_decision_tree_classifier.score(X_test, y_test)))

decision_tree_best_params = grid_search_decision_tree_classifier.best_params_
print("Decision Tree best params: " + str(decision_tree_best_params))

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Decision Tree best grid score: 0.1615495474917537
Decision Tree grid test score: 0.1622821321158097
Decision Tree best params: {'criterion': 'entropy', 'max_depth': 30, 'random_state': 4, 'splitter': 'random'}


In [31]:
# Predicting the Y_test values using the model from gridsearch
y_pred = grid_search_decision_tree_classifier.predict(X_test)

In [32]:
# Generating the classification_report
grid_search_decision_tree_classification_reprt = classification_report(y_test, y_pred)
print("Decision Tree Classification report with whole data")
print(grid_search_decision_tree_classification_reprt)

Decision Tree Classification report with whole data
              precision    recall  f1-score   support

        11.0       0.82      0.32      0.47       757
        14.0       0.00      0.00      0.00         1
        20.0       0.00      0.00      0.00         1
        21.0       0.00      0.00      0.00         6
        22.0       0.00      0.00      0.00         6
        24.0       0.00      0.00      0.00         3
        25.0       0.30      0.25      0.27        12
        26.0       0.00      0.00      0.00         5
        27.0       0.40      0.30      0.34        20
        28.0       0.40      0.43      0.41        14
        29.0       0.16      0.23      0.19        13
        30.0       0.16      0.07      0.10        54
        31.0       0.37      0.12      0.18        59
        32.0       0.19      0.16      0.17       127
        33.0       0.22      0.30      0.25       172
        34.0       0.14      0.16      0.15       277
        35.0       0.75      

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [33]:
# Selecting features using RandomForestClassifier
select = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold='median')
select.fit(X_train, y_train)
X_train_selected = select.transform(X_train)
X_test_selected = select.transform(X_test)

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/feature_selection/_from_model.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


In [34]:
# Applying DecisionTreeClassifier using the best params from the grid search and with selected data
decision_tree_classifier = DecisionTreeClassifier(criterion = decision_tree_best_params['criterion'],\
    max_depth = decision_tree_best_params['max_depth'],\
    random_state = decision_tree_best_params['random_state'],\
    splitter = decision_tree_best_params['splitter'])

In [12]:
decision_tree_classifier.fit(X_train_selected, y_train)
y_pred = decision_tree_classifier.predict(X_test_selected)
decision_tree_classification_reprt = classification_report(y_test, y_pred)
print("Decision Tree Classification report with selected data")
print(decision_tree_classification_reprt)

Decision Tree Classification report with selected data
              precision    recall  f1-score   support

        11.0       1.00      0.31      0.47       757
        14.0       0.00      0.00      0.00         1
        20.0       0.00      0.00      0.00         1
        21.0       0.00      0.00      0.00         6
        22.0       0.00      0.00      0.00         6
        24.0       0.00      0.00      0.00         3
        25.0       0.00      0.00      0.00        12
        26.0       0.00      0.00      0.00         5
        27.0       0.00      0.00      0.00        20
        28.0       0.00      0.00      0.00        14
        29.0       0.00      0.00      0.00        13
        30.0       0.00      0.00      0.00        54
        31.0       0.00      0.00      0.00        59
        32.0       0.00      0.00      0.00       127
        33.0       0.00      0.00      0.00       172
        34.0       0.00      0.00      0.00       277
        35.0       0.00   

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [40]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier


## KNN

In [4]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,20))
n_neighbors = list(range(1,10))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=5)
#Fit the model
best_model = clf.fit(X_train, y_train)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWa

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWa

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWa

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWa

...

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWa

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWa

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWa

Best leaf_size: 1
Best p: 1
Best n_neighbors: 9


/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [5]:
print("KNN best grid score: " + str(clf.best_score_))
print("KNN grid test score: " + str(clf.score(X_test, y_test)))

decision_tree_best_params = clf.best_params_
print("KNN best params: " + str(decision_tree_best_params))

KNN best grid score: 0.12055843483907377
KNN grid test score: 0.12485052723122078
KNN best params: {'leaf_size': 1, 'n_neighbors': 9, 'p': 1}


In [7]:
KNNclassifier = KNeighborsClassifier(leaf_size = 1, n_neighbors = 9, p = 1)
KNNclassifier.fit(X_train_selected, y_train)
y_pred = KNNclassifier.predict(X_test_selected)
KNN_classification_reprt = classification_report(y_test, y_pred)
print("Decision Tree Classification report with selected data")
print(KNN_classification_reprt)

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Decision Tree Classification report with selected data
              precision    recall  f1-score   support

        11.0       0.06      0.39      0.10       757
        14.0       0.00      0.00      0.00         1
        20.0       0.00      0.00      0.00         1
        21.0       0.00      0.00      0.00         6
        22.0       0.00      0.00      0.00         6
        24.0       0.00      0.00      0.00         3
        25.0       0.00      0.00      0.00        12
        26.0       0.00      0.00      0.00         5
        27.0       0.00      0.00      0.00        20
        28.0       0.00      0.00      0.00        14
        29.0       0.00      0.00      0.00        13
        30.0       0.00      0.00      0.00        54
        31.0       0.00      0.00      0.00        59
        32.0       0.01      0.02      0.01       127
        33.0       0.00      0.01      0.00       172
        34.0       0.01      0.06      0.01       277
        35.0       0.00   

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

## Logistic regression

In [3]:
model = LogisticRegression()
solvers = ['newton-cg','liblinear']
penalty = ['l2', 'l1']
c_values = [0.1, 1, 10,  100]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = 3
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs = -1, cv=cv, scoring='accuracy', verbose = 10 )
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
12 fits failed out of a total of 48.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sharanbasavasumbad/pyth

Best: 0.054277 using {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


In [5]:
print("Logistic Regression best grid score: " + str(grid_result.best_score_))
print("Logistic Regression grid test score: " + str(grid_result.score(X_test, y_test)))

LR_best_params = grid_result.best_params_
print("Logistic Regression best params: " + str(LR_best_params))

Logistic Regression best grid score: 0.05427692881103243
Logistic Regression grid test score: 0.05292241910352575
Logistic Regression best params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}


In [8]:
LRclassifier = LogisticRegression(C= 0.1, penalty = 'l2', solver = 'liblinear')
LRclassifier.fit(X_train_selected, y_train)
y_pred = LRclassifier.predict(X_test_selected)
LR_classification_reprt = classification_report(y_test, y_pred)
print("Log Reg Tree Classification report with selected data")
print(LR_classification_reprt)

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Log Reg Tree Classification report with selected data
              precision    recall  f1-score   support

        11.0       0.07      0.33      0.12       757
        14.0       0.00      0.00      0.00         1
        20.0       0.00      0.00      0.00         1
        21.0       0.00      0.00      0.00         6
        22.0       0.00      0.00      0.00         6
        24.0       0.00      0.00      0.00         3
        25.0       0.00      0.00      0.00        12
        26.0       0.00      0.00      0.00         5
        27.0       0.00      0.00      0.00        20
        28.0       0.00      0.00      0.00        14
        29.0       0.00      0.00      0.00        13
        30.0       0.00      0.00      0.00        54
        31.0       0.00      0.00      0.00        59
        32.0       0.00      0.00      0.00       127
        33.0       0.00      0.00      0.00       172
        34.0       0.00      0.00      0.00       277
        35.0       0.00    

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

## SVC

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid_SVC = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)


# fitting the model for grid search
grid_SVC.fit(X_train, y_train)

In [15]:
print("SVM best grid score: " + str(grid_SVC.best_score_))
print("SVM grid test score: " + str(grid_SVC.score(X_test, y_test)))

Svm_best_params = grid_SVC.best_params_
print("Logistic Regression best params: " + str(Svm_best_params))

SVM best grid score: 0.064572762
SVM grid test score: 0.06321784
Logistic Regression best params: {'C': 1, 'gamma': 'l2', 'kernel': 'linear'}


In [13]:
from sklearn import svm
modelSVM = svm.SVC(kernel='linear', gamma =0.1, C = 1, probability=True)
modelSVM.fit(X_train_selected, y_train)
y_pred = modelSVM.predict(X_test_selected)
SVM_classification_reprt = classification_report(y_test, y_pred)
print("SVM  Classification report with selected data")
print(SVM_classification_reprt)

SVM  Classification report with selected data
              precision    recall  f1-score   support

        11.0       0.47      0.31      0.38       757
        14.0       0.00      0.00      0.00         1
        20.0       0.00      0.00      0.00         1
        21.0       0.00      0.00      0.00         6
        22.0       0.00      0.00      0.00         6
        24.0       0.00      0.00      0.00         3
        25.0       0.00      0.00      0.00        12
        26.0       0.00      0.00      0.00         5
        27.0       0.00      0.00      0.00        20
        28.0       0.00      0.00      0.00        14
        29.0       0.00      0.00      0.00        13
        30.0       0.00      0.00      0.00        54
        31.0       0.00      0.00      0.00        59
        32.0       0.00      0.00      0.00       127
        33.0       0.00      0.00      0.00       172
        34.0       0.00      0.00      0.00       277
        35.0       0.00      0.00  

/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sharanbasavasumbad/python-env/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

**Based on all the classofication report and best score out of the four models Decisoin Tree , KNN , SVC and Loistic Regression , Decision Tree With parameters  {'criterion': 'entropy', 'max_depth': 30, 'random_state': 4, 'splitter': 'random'} Performs the best among all**